<a href="https://colab.research.google.com/github/Griznt/colab_test_tasks/blob/main/tast_task_magnit_it_%F0%9F%A7%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

Есть таблица с товарами: items

```
item_id    name                 price       update_date
-------    ----------------     ------      -----------
1          Ручка гелевая        10          01-02-2020
2          Карандаш 1HH         3           01-01-2020
1          Ручка шариковая      10          01-03-2020
3          Ластик               5           01-07-2020
2          Карандаш 1HH         2           01-04-2020
1          Ручка шариковая      5           01-05-2020
2          Карандаш 1H          7           01-06-2020
```


И таблица с заказами: orders


```
order_id    user_id     item_id     order_date
-------    --------     -------     ----------
1           1           1           01-02-2020
2           2           2           01-05-2020
3           1           3           01-07-2020
4           3           2           01-07-2020
5           2           1           01-04-2020
6           1           1           01-06-2020
```

Написать запрос, который:
1. Выведет актуальное состояние товаров на 01-06-2020
2. Товары, купленные по цене больше или равно чем 3















In [ ]:
!gdown 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
100% 6.50M/6.50M [00:00<00:00, 263MB/s]


In [ ]:
!gdown 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi -O /usr/lib/python3.7/lib-dynload/
# MENU: Runtime > Restart runtime
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /usr/lib/python3.7/lib-dynload/_sqlite3.cpython-37m-x86_64-linux-gnu.so
100% 6.50M/6.50M [00:00<00:00, 223MB/s]


In [1]:
import sqlite3
import pandas as pd

In [2]:
sqlite3.sqlite_version  # '3.38.0'

'3.38.0'

In [3]:
con = sqlite3.connect('db')

In [4]:
def select(sql):
  return pd.read_sql(sql, con)

Заполним БД тестовыми данными.

In [5]:
cur = con.cursor()

In [6]:
sql = '''
drop table if exists items;

create table items (item_id char(1),  name char(255), price int, update_date Date);

insert into items values  
(1,              'Ручка гелевая',            10,              '2020-02-01'),
(2,              'Карандаш 1HH',             3,              '2020-01-01'),
(1,              'Ручка шариковая',          10,              '2020-03-01'),
(3,              'Ластик',                   5,               '2020-07-01'),
(2,              'Карандаш 1HH',             2,               '2020-04-01'),
(1,              'Ручка шариковая',          5,              '2020-05-01'),
(2,              'Карандаш 1H',              7,               '2020-06-01')

'''


In [7]:
cur.executescript(sql)

In [8]:
sql = '''
select t.* from items t
'''

In [9]:
select(sql)

,item_id,name,price,update_date
0,1,Ручка гелевая,10,2020-02-01
1,2,Карандаш 1HH,3,2020-01-01
2,1,Ручка шариковая,10,2020-03-01
3,3,Ластик,5,2020-07-01
4,2,Карандаш 1HH,2,2020-04-01
5,1,Ручка шариковая,5,2020-05-01
6,2,Карандаш 1H,7,2020-06-01


In [86]:
sql = '''
drop table if exists orders;

create table orders (order_id int,  user_id int, item_id int, order_date Date);

insert into orders values  
(1,               1,               1,               '2020-02-01'),
(2,               2,               2,               '2020-05-01'),
(3,               1,               3,               '2020-07-01'),
(4,               3,               2,               '2020-07-01'),
(5,               2,               1,               '2020-04-01'),
(6,               1,               1,               '2020-06-01')

'''

In [87]:
con.executescript(sql)

In [12]:
sql = '''
select t.* from orders t
'''

In [13]:
select(sql)

,item_id,name,price,update_date
0,1,1,1,2020-02-01
1,2,2,2,2020-05-01
2,3,1,3,2020-07-01
3,4,3,2,2020-07-01
4,5,2,1,2020-04-01
5,6,1,1,2020-06-01


Написать запрос, который:
1. Выведет актуальное состояние товаров на 01-06-2020

In [132]:
sql = '''
with items_upd as (
select 
 t.*,
 case when JULIANDAY(t.update_date) - JULIANDAY('2020-06-01') <= 0 then t.price
 else null end as actual_max_price,
 row_number() over (partition by t.item_id order by t.update_date desc) as max_price_position

from items t
order by t.item_id asc, JULIANDAY(t.update_date) - JULIANDAY('2020-06-01') desc
)

select t.*
 from items_upd t
where t.max_price_position = 1
'''

In [133]:
select(sql)

,item_id,name,price,update_date,actual_max_price,max_price_position
0,1,Ручка шариковая,5,2020-05-01,5.0,1
1,2,Карандаш 1H,7,2020-06-01,7.0,1
2,3,Ластик,5,2020-07-01,NaN,1


In [80]:
sql = '''with items_upd as (
select 
    t.* 
from items t
where julianday(t.update_date) - julianday('2020-06-01') >= 0
),
items_max as (
select 
    t.*,
    max(t.update_date) over(partition by t.name order by t.update_date desc) as update_date_max,
    t.update_date - max(t.update_date) over(partition by t.name order by t.update_date desc) as date_diff
from items_upd t
)
select
    t.name,
    t.price,
    t.update_date
from items_max t
where t.date_diff = 0
'''

In [81]:
select(sql)

,name,price,update_date
0,Карандаш 1H,7,2020-06-01
1,Ластик,5,2020-07-01


Написать запрос, который:
2. Товары, купленные по цене больше или равно чем 3


In [150]:
sql = '''
with items_upd as (
select 
  t.*, 
  i.name,
  i.price,
  i.update_date
from orders t
join items i on t.item_id = i.item_id and i.update_date <= t.order_date
order by t.order_id, i.update_date desc
),
orders_with_actual_price as (
select 
  t.*,
  row_number() over(partition by t.order_id order by t.update_date desc) as rnk
from items_upd t
)

select 
  t.*
from orders_with_actual_price t
where t.rnk = 1 and t.price >= 3
'''   

In [151]:
select(sql)

,order_id,user_id,item_id,order_date,name,price,update_date,rnk
0,1,1,1,2020-02-01,Ручка гелевая,10,2020-02-01,1
1,3,1,3,2020-07-01,Ластик,5,2020-07-01,1
2,4,3,2,2020-07-01,Карандаш 1H,7,2020-06-01,1
3,5,2,1,2020-04-01,Ручка шариковая,10,2020-03-01,1
4,6,1,1,2020-06-01,Ручка шариковая,5,2020-05-01,1


-----------------------------------------------------
Определить максимальную последовательность одинаковых символов в строке
Пример строки:
s = 'sssffdddewvffffdddd'

In [ ]:
s = 'sssffdddewvffff'

In [ ]:
result = {}

for letter in s:
  max = 0
  counter = 0
  for i in s:
    if letter == i:
      counter += 1
    else:
      if max < counter:
        max = counter
      counter = 0
  if max < counter:
    max = counter
  result[letter] = max

print(result)

{'s': 3, 'f': 4, 'd': 3, 'e': 1, 'w': 1, 'v': 1}


Мы играем в игру. Можно бросить кубик и какое число выпадет, столько тысяч рублей мы выиграем. Если нас не устроил первый бросок, мы можем перекинуть один раз.  
Нужно определить наиболее выигрышную стратегию

In [ ]:
cube = pd.DataFrame([1,2,3,4,5,6])

Нам выпало число. Если мы кинем кубик и выпадет столько же или больше, то перекидывать стоит. 

In [ ]:
result = []

Будем повторять эксперимент много раз. Какая стратегия наиболее выигрышная?

Кидаем один раз

In [ ]:
n = 10000
for i in range(n):
  t1 = cube.sample(1)
  result.append(t1)

np.mean(result) * n

38568.90476190476

Перекидываем, если выпало меньше среднего



In [ ]:
n = 10000
for i in range(n):
  t1 = cube.sample(1)
  if t1.values[0][0] < np.mean(cube).values[0]:
    t1 = cube.sample(1)
  result.append(t1)

np.mean(result) * n

38746.681818181816

In [ ]:
n = 10000
for i in range(n):
  t1 = cube.sample(1)
  if t1.values[0][0] < 4:
    t1 = cube.sample(1)
  result.append(t1)

np.mean(result) * n

39164.153846153844